In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('../data/Raptor Contract.docx.pdf')
pages = loader.load_and_split()

In [2]:
len(pages)

76

In [3]:
length = 0
count = 0
for page in pages:
    length += len(page.page_content)
    count += 1

length, count

(225087, 76)

In [4]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter

CHUNK_SIZE = 300
CHUNK_OVERLAP = 30

text_splitter = CharacterTextSplitter(chunk_size = CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
docs = text_splitter.split_documents(pages)

In [5]:
print(len(docs[0].page_content))

801


In [6]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Qdrant

embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

qdrant = Qdrant.from_documents(
    docs, 
    embeddings,
    path='local_qdrant',
    collection_name="contract_documents",
    force_recreate=True,
)

/home/misge/miniconda3/envs/wk-11/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [11]:
query = 'Under what circumstances and to what extent the Sellers are responsible for a breach of representations and warranties?'

found_docs = qdrant.similarity_search(query, k=2)

print(found_docs[0].page_content)

any
Seller .
Without
limiting
the
generality
of
the
foregoing,
the
Sellers’
Representative
is
hereby 
authorized,
on
behalf
of
the
Sellers,
to:
(i)
in
connection
with
the
Closing,
execute
and
receive
all
documents,
instruments, 
certificates,
statements
and
agreements
on
behalf
of
and
in
the
name
of
each
Seller
necessary
to 
effectuate
the
Closing
and
consummate
the
Contemplated
Transactions;
(ii)
receive
and
give
all
notices
and
service
of
process,
make
all
filings,
enter
into
all 
Contractual
Obligations,
make
all
decisions,
bring,
prosecute,
defend,
settle,
compromise
or 
otherwise
resolve
all
claims,
disputes
and
Actions,
authorize
payments
in
respect
of
any
such 
claims,
disputes
or
Actions,
and
take
all
other
actions,
in
each
case,
with
respect
to
the
matters 
set
forth
in
Section
2.06
,
Article
VII
,
Article
VIII
or
any
other
Actions
directly
or
indirectly 
arising
out
of
or
relating
to
this
Agreement,
the
Escrow
Agreement
or
the
Contemplated 
Transactions;
(iii)
receive
and
giv

In [12]:
print(len(found_docs[0].page_content))

3506


### Generation

### load_qa_chain


In [13]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAIChat

chain = load_qa_chain(OpenAIChat(temperature=0), chain_type='stuff')

answer = chain.run(input_documents=found_docs, question=query)

/home/misge/miniconda3/envs/wk-11/lib/python3.12/site-packages/langchain_community/llms/openai.py:1070: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


In [14]:
answer

'The Sellers are responsible for a breach of representations and warranties to the extent that they indemnify the Sellers’ Representative from any Losses arising out of its serving as the Sellers’ Representative, except for Losses arising out of or caused by the Sellers’ Representative’s gross negligence, bad faith, or willful misconduct. Each Seller will severally (and not jointly) indemnify the Sellers’ Representative from any Losses.'

#### load_qa_with_sources_chain

In [15]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

chain = load_qa_with_sources_chain(OpenAIChat(temperature=0.2, model_name='gpt-3.5-turbo'), chain_type='stuff')

answer = chain({'input_documents': found_docs, 'question': query}, return_only_outputs=True)

/home/misge/miniconda3/envs/wk-11/lib/python3.12/site-packages/langchain_community/llms/openai.py:1070: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
/home/misge/miniconda3/envs/wk-11/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [20]:
answer 

{'answer': "The Sellers' Representative shall not be held liable by any of the Sellers for actions or omissions in exercising or failing to exercise all or any of the power and authority of the Sellers' Representative pursuant to this Agreement, except in the case of the Sellers' Representative's gross negligence, bad faith, or willful misconduct. Each Seller will severally (and not jointly) indemnify (in accordance with its Pro Rata Percentage) the Sellers' Representative from any Losses arising out of its serving as the Sellers' Representative hereunder, except for Losses arising out of or caused by the Sellers' Representative's gross negligence, bad faith, or willful misconduct. The Sellers' Representative is serving in its capacity as such solely for purposes of administrative convenience and is not personally liable in such capacity for any of the obligations of the Sellers hereunder. The Buyer agrees that it will not look to the personal assets of the Sellers' Representative, act

#### RetrieverQA

In [16]:
from langchain.chains import RetrievalQA 
from langchain.chains import RetrievalQAWithSourcesChain 

qa = RetrievalQA.from_chain_type(llm=OpenAIChat(temperature=0.2, model_name='gpt-3.5-turbo'), chain_type='stuff', retriever=qdrant.as_retriever())

answer = qa.run(query)

/home/misge/miniconda3/envs/wk-11/lib/python3.12/site-packages/langchain_community/llms/openai.py:1070: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


In [17]:
answer 

'The Sellers are responsible for a breach of representations and warranties to the extent that they indemnify the Sellers’ Representative from any Losses arising out of its service as the Sellers’ Representative, except for Losses arising out of or caused by the Sellers’ Representative’s gross negligence, bad faith, or willful misconduct. Each Seller will severally (and not jointly) indemnify the Sellers’ Representative in accordance with its Pro Rata Percentage.'

#### RetrieverQAWithSourcesChain

In [18]:
chain = RetrievalQAWithSourcesChain.from_chain_type(OpenAIChat(temperature=0.2, model_name='gpt-3.5-turbo'), chain_type='stuff', retriever=qdrant.as_retriever())

answer = chain({'question': query}, return_only_outputs=True)


/home/misge/miniconda3/envs/wk-11/lib/python3.12/site-packages/langchain_community/llms/openai.py:1070: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


In [19]:
answer

{'answer': "The Sellers' Representative shall not be held liable by any of the Sellers for actions or omissions in exercising or failing to exercise all or any of the power and authority of the Sellers' Representative pursuant to this Agreement, except in the case of the Sellers' Representative's gross negligence, bad faith, or willful misconduct. Each Seller will severally (and not jointly) indemnify (in accordance with its Pro Rata Percentage) the Sellers' Representative from any Losses arising out of its serving as the Sellers' Representative hereunder, except for Losses arising out of or caused by the Sellers' Representative's gross negligence, bad faith, or willful misconduct. The Sellers' Representative is serving in its capacity as such solely for purposes of administrative convenience and is not personally liable in such capacity for any of the obligations of the Sellers hereunder. The Buyer agrees that it will not look to the personal assets of the Sellers' Representative, act